# Module 2 Assignment: Adding functionalities to your Automatic Code Review Crew

Welcome to Module 2's assignment! In this lab, you will build upon the Automatic Code Review Crew you created in the assignment in Module 1. This time, you will add the new functionalities you learned in this lesson to take your crew's performance to the next level.

## Background
Your first try at the automation tool for code reviewing was a success. You want to go one step further by adding memory, guardrails and execution hooks to ensure you get the best possible results out of your tool.

Here is a visual summary of the structure of your crew, as well as the new elements you will be adding: 
<div style="text-align: center;">
<img src="./images/agents-tasks-diagram.png" width=600>
</div>

## General instructions for grading
- Replace all `None` instances with your own solution.
- You can add new cells to experiment, but these will be omitted by the grader. Only use the provided cells for your solution code.
- Before submitting, make sure all the cells in your lab work correctly.
- **Do not change variable names**: if you modify variable names, the grader won't be able to find your solutions
- **Use the provided configuration**: for grading, please use all provided configurations. Don't change the configuration files or settings. You can experiment after submitting your lab.
- To submit your notebook, save it and then click on the red **Submit Assignment** button at the top right of the page.

**<font color='#5DADEC'>Please make sure to save your work periodically, so you don't lose any progress.</font>**

## Table of contents

1. [Set up your notebook](#1)
2. [Agents](#2)
3. [Guardrails](#3)
   - [Exercise 1: Write the `security_review_output_guardrail` guardrail](#ex1)
   - [Exercise 2: Write the `review_decision_guardrail` guardrail](#ex2)
4. [Tasks](#4)
   - [Exercise 3: Create the Analyze Code Quality task](#ex3)
   - [Exercise 4: Create the Review Security task](#ex4)
   - [Exercise 5: Create the Review Decision task](#ex5)
5. [Execution Hooks](#5)
   - [Exercise 6: Create a hook to read the PR file](#ex6)
6. [Creating the Crew](#6)
   - [Exercise 7: Define the Crew](#ex7)
   - [Exercise 8: Kickoff the Crew](#ex8)

<a id="1"></a>

## 1 - Set up your notebook

Begin by importing all necessary modules and configure your environment variables to connect to the LLM APIs.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

`!pip install crewai[tools]=1.3.0`

<a id="1-1"></a>

### 1.1 - Import modules
Run the following cell to import all the modules you will need for this lab.

In [1]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel
from utils import get_openai_api_key, get_serper_api_key, clean_markdown
from IPython.display import Markdown, display
import yaml
import os
os.environ["CREWAI_TESTING"] = "true"

In [3]:
import unittests
import dill

<a id="1-2"></a>

### 1.2 - Setup the environment variables

Next, set up the environment variables to connect to the APIs, and create the LLM instance you will use for your Agents

In [4]:
import os
os.environ["CREWAI_TESTING"] = "true"
from utils import get_openai_api_key

# set up the OpenAI API key 
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
# set up the OpenAI model to use
os.environ["MODEL"] = 'gpt-4o-mini'

<a id="1-3"></a>

### 1.3 - Import configuration files

Since you already defined the agents and tasks in Module 1's assignment, this time you will just load the parameters from a `YAML` file. These files contain `role`, `goal` and `backstory` for Agents, and `description` and `expected_output` for Tasks. This way, you only need to set up the new parameters and functionalities.

Run the next cell to define the configurations.

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

<a id="2"></a>

## 2 - Agents

You will begin by creating your agents. This time you don't have to write any code, because you will be using exactly the same agents as in the previous assignment. The only difference is that this time the `role`, `goal` and `backstory` for each agent are given to you in the YAML configuration file. 

Run the cell below to create all the agent instances

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
# get the Serper API key
from utils import get_serper_api_key
serper_api_key = get_serper_api_key()

### START CODE HERE ###
# create the instance of the SerperDevTool. Set the search_url to "https://owasp.org"
serper_search_tool = SerperDevTool(search_url="https://owasp.org")

# create the instance of the ScrapeWebsiteTool, which does not need any arguments
scrape_website_tool = ScrapeWebsiteTool()

In [8]:


# create the Senior Developer agent
senior_developer = Agent( 
    # load role, goal, and backstory from the YAML configuration
    config=agents_config['senior_developer'],
    # set verbose
    verbose=True 
)

# create the Security Engineer agent
security_engineer = Agent( 
    # load role, goal, and backstory from the YAML configuration
    config=agents_config['security_engineer'],
    # add the website search tools (you need to unpack the list)
    tools=[serper_search_tool, scrape_website_tool],
    # set verbose 
    verbose=True,
)

# create the Tech Lead agent
tech_lead = Agent(
    # load role, goal, and backstory from the YAML configuration
    config=agents_config['tech_lead'],
    # set verbose
    verbose=True
)

<a id="3"></a>

## 3 - Guardrails
You need to guarantee security reviews have proper structure and standardized risk levels to prevent vulnerabilities from being misclassified or overlooked. Guardrails provide this essential validation!

You will define two guardrails:


| Name             | Functionality      | Task          |
|------------------|--------------------|---------------|
| `security_review_output_guardrail`| Ensures the security risks are within the specified categories| `review_security`|
| `review_decision_guardrail` | 	Ensures output includes an actionable decision (approve, etc.) | `make_review_decision`|

Each guardrail must return a tuple with a `bool` of whether the checks passed or failed, and a message (optional). If you want to know more details about guardrails in CrewAI, please check the [docs](https://docs.crewai.com/en/concepts/tasks#task-guardrails).

<a id="ex1"></a>

### Exercise 1: Write the `security_review_output_guardrail` guardrail

Complete the cell below to define the function to create the guardrail for the security review output. This guardrail needs to validate: 
- The `risk_level` for each vulnerability is one of the three accepted categories: `low`, `medium` or `high`.
- The `highest_risk` actually matches the highest value in `risk_level`.

Some of the structure is already given to you; you only need to fill in the `None` placeholders.

In [25]:
# GRADED CELL: Exercise 1

def security_review_output_guardrail(output):
    
    # get the (JSON) output from the TaskOutput object
    try: 
        json_output = output if type(output)==dict else output.json_dict 
    except Exception as e:
        return (False, ("Error retrieving the `json_dict` argument: "
                        f"\n{str(e)}\n"
                        "Make sure you set the output_json parameter in the Task."
                        )
                )

    # if the task returned no JSON, fail gracefully
    if not json_output:
        return (False, "Missing JSON output from task.")

    # define risk levels
    valid_risk_levels = ['low', 'medium', 'high']
    
    # Check if security_vulnerabilities key exists
    if 'security_vulnerabilities' not in json_output:
        return (False, f"Missing 'security_vulnerabilities' key in output. Got keys: {list(json_output.keys())}")
    
    # validate that each of the risk levels has a valid value
    for vuln in json_output['security_vulnerabilities']:
        # validate the risk level
        if vuln['risk_level'].lower() not in valid_risk_levels: 
            error_message = f"Invalid risk level: {vuln['risk_level']}"
            return (False, error_message) 
    
    # validate that the highest risk level matches the highest risk level in the vulnerabilities
    if 'highest_risk' not in json_output:
        return (False, "Missing 'highest_risk' key in output.")

    # if the highest risk level is not valid risk level, return an error message
    if json_output["highest_risk"].lower() not in valid_risk_levels:
        error_message = f"Invalid highest risk level: {json_output['highest_risk']}"
        return (False, error_message)
    
    # if it is one of the valid risk levels, then check if it matches the highest 
    # risk level in the vulnerabilities
    else:
        # get all risk_level values
        risk_levels = [vuln['risk_level'].lower() for vuln in json_output['security_vulnerabilities']] 
        
        # if "high" in risk levels, then highest risk level should be high
        if "high" in risk_levels: 
            if json_output["highest_risk"].lower() != "high":
                error_message = "Highest risk level does not match the highest risk level in the vulnerabilities." 
                return (False, error_message)
            
        # if high is not present and medium is in risk levels, then highest risk level should be medium
        elif "medium" in risk_levels: 
            if json_output["highest_risk"].lower() != "medium":
                error_message = "Highest risk level does not match the highest risk level in the vulnerabilities." 
                return (False, error_message)
            
        # if high and medium are not present, then lowest risk level should be low
        elif "low" in risk_levels: 
            if json_output["highest_risk"].lower() != "low":
                error_message = "Highest risk level does not match the highest risk level in the vulnerabilities." 
                return (False, error_message)
    
    return (True, output.json_dict)


Try the guardrail with an invalid JSON dictionary, where the highest risk is not correctly identified.

In [26]:
# create the output json for testing
invalid_json = {"highest_risk": "medium", 
                        "security_vulnerabilities": [{"risk_level": "high"}, 
                                                     {"risk_level": "medium"}]}
# test the guardrail
security_review_output_guardrail(invalid_json)

(False,
 'Highest risk level does not match the highest risk level in the vulnerabilities.')

##### **Expected output**:
```
(False,
 'Highest risk level does not match the highest risk level in the vulnerabilities.')
```

In [ ]:
# test the guardrail
unittests.test_security_review_output_guardrail(security_review_output_guardrail)

<a id="ex2"></a>

### Exercise 2: Write the `review_decision_guardrail` guardrail

Complete the cell below to define the function to create the guardrail for the review decision output. This guardrail needs to make sure the output includes one of the required decision values: "approve", "request changes" or "escalate".

Some of the structure is already given to you; you only need to fill in the `None` placeholders.

In [27]:
# GRADED CELL: Exercise 2

def review_decision_guardrail(output):
    # get the raw output from the TaskOutput object
    try:
        output = output if type(output)==str else output.raw
    except Exception as e:
        return (False, ("Error retrieving the `raw` argument: "
                        f"\n{str(e)}\n"
                        "Make sure you set the raw parameter in the Task."
                        )
                )

    # define the keywords to check for in the output
    keywords = ["approve", "request changes", "escalate"]
    
    ### START CODE HERE ###

    # check if any of the keywords are present in the output
    if not any(keyword in output.lower() for keyword in keywords):
        # write a suitable message to the console
        error_message = "Output does not include one of the valid actionable decisions."
        return (False, error_message)

    # if all checks pass, return True, and the output
    return (True, output)

    ### END CODE HERE ###


Try the guardrail with an output missing the possible decision values.

In [28]:
# create the raw output for testing
input = 'Final decision: Elevate to human'

# test the guardrail
review_decision_guardrail(input)

(False, 'Output does not include one of the valid actionable decisions.')

##### **Expected output**:
In this case, the message will vary depending on what you chose
```
(False, 'Output does not include one of the valid actionable decisions.')
```

In [ ]:
# test the guardrail
unittests.test_review_decision_guardrail(review_decision_guardrail)

<a id="4"></a>

## 4 - Tasks
Now that you have defined the guardrails and execution hooks, you are ready to define the tasks. You will create the three original tasks. For each one, you will load the `description`, `expected_output` and `name` from the yaml file, but you will need to add the guardrails and output types. The difference file (PR) contents will be set in the `file_content` key of the inputs to the crew. 

<a id="ex3"></a>

### Exercise 3: Create the Analyze Code Quality task
Start by defining the Analyze Code Quality task. The structure of the JSON output is defined for you. It includes the following keys:
* `critical_issues`: a list of issues that need fixing
* `minor_issues`: a list of suggested improvements
* `reasoning`: text with the explanation of the reasoning

You should:
1. Define the Task:
    * Use the `config` parameter to load the task configuration from the YAML file
    * Set the `output_json` parameter with the corresponding structure
    * Assign to the corresponding `agent`

In [29]:
# GRADED CELL: Exercise 3

# Define the pydantic model for the code quality analysis output
class CodeQualityJSON(BaseModel):
    critical_issues: list[str]
    minor_issues: list[str]
    reasoning: str

### START CODE HERE ###

# Create the quality analysis task
analyze_code_quality = Task(
    # Load the expected output, and name from the YAML configuration
    config=tasks_config['analyze_code_quality'], 
    # Define the output type as a pydantic model
    output_json=CodeQualityJSON,
    # Define the agent that will perform this task
    agent=senior_developer, 
)

### END CODE HERE ###


In [ ]:
# test the analyze_code_quality task
unittests.test_analyze_code_quality(analyze_code_quality)

<a id="ex4"></a>

### Exercise 4: Create the Review Security task

1. Define the JSON structure of the Security vulnerabilities. You need the following keys:
    * `description`: string with the description
    * `risk_level`: a string indicating the level
    * `evidence`: a string showing the evidence for the risk

2. Define the JSON structure for the output. You need the following keys:
    * `security_vulnerabilities`: a list of Security vulnerabilities (structure defined before)
    * `blocking`: a boolean indicating if security issues should block approval
    * `highest_risk`: a string with the most severe risk level found
    * `security_recommendations`: a list of strings with specific fixes for identified vulnerabilities

2. Define the Task:
    * Use the `config` parameter to load the task configuration from the YAML file
    * Set the `output_json` parameter
    * Add the guardrail
    * Assign to the corresponding agent

In [30]:
# GRADED CELL: Exercise 4

### START CODE HERE ###

# Define the pydantic model for the security vulnerabilities
class SecurityVulnerability(BaseModel):
    description: str
    risk_level: str
    evidence: str

# Define the pydantic model for the security review output
class ReviewSecurityJSON(BaseModel):
    security_vulnerabilities: list[SecurityVulnerability] 
    blocking: bool
    highest_risk: str
    security_recommendations: list[str]

# Create the security review task
review_security = Task(
    # Load the expected output, agent, and name from the YAML configuration
    config=tasks_config['review_security'], 
    # Define the output type as a pydantic model
    output_json=ReviewSecurityJSON,
    # Add the security_review_output_guardrail guardrail
    guardrails=[security_review_output_guardrail],
    # Define the agent that will perform this task
    agent=security_engineer 
)

### END CODE HERE ###


In [ ]:
# test the review_security task
unittests.test_review_security(review_security, SecurityVulnerability, ReviewSecurityJSON)

<a id="ex5"></a>

### Exercise 5: Create the Review Decision task

Next, define the Review Decision task. For this task, you will need to set the corresponding `guardrail`. 

1. Define the Task:
    * Use the `config` parameter to load the task configuration from the YAML file
    * Set the `Markdown` parameter, to get the final report in Markdown format
    * Add the guardrail
    * Assign to the corresponding agent

In [31]:
# GRADED CELL: Exercise 5

### START CODE HERE ###

# Create the review decision task
make_review_decision = Task(
    # Load the expected output, agent, and name from the YAML configuration
    config=tasks_config['make_review_decision'], 
    # Set the Markdown parameter to get the final report in Markdown format
    markdown=True,
    # Add the review_decision_guardrail guardrail
    guardrails=[review_decision_guardrail],
    # Set the context with the previous task objects
    context=[analyze_code_quality, review_security], 
    # Define the agent that will perform this task
    agent=tech_lead 
)

### END CODE HERE ###


In [ ]:
# test the make_review_decision task
unittests.test_make_review_decision(make_review_decision)

<a id="5"></a>

## 5 - Execution Hooks

All your tasks require access to the code difference (PR file), but reading the file isn't really an "agentic" task that requires intelligent decision-making. Instead of having your agents read the file directly, you can create a **before-kickoff hook** that handles this automatically. This hook will read the file and add its content to the crew's inputs before the agents begin their work.

<a id="ex6"></a>

### Exercise 6: Create a hook to read the PR file
Write a function to be used as a hook. This function should:
1. Receive the file path from the `file_path` parameter in the crew's inputs
2. Read the contents of the PR file
3. Add a new key to the `input` dictionary called `file_content` containing the loaded file contents

In [32]:
# GRADED CELL: Exercise 6

# Define the execution hook to read the PR file
def read_file_hook(inputs):
    
    ### START CODE HERE ###
    
    # get the file_path from the inputs
    filename = inputs.get("file_path")
    
    ### END CODE HERE ###

    # if the filename is not provided, raise an error
    if not filename:
        raise ValueError("Missing 'file_path' in inputs")

    # try reading the file
    try:
        with open(filename, "r") as f:
            file_contents = f.read()
    # if there are any issues, raise an error
    except Exception as e:
        raise RuntimeError(f"Failed to read file {filename}: {e}")

    
    ### START CODE HERE ###

    # add the file contents to the inputs
    inputs["file_content"] = file_contents
    
    # return the modified inputs
    return inputs

    ### END CODE HERE ###


In [ ]:
# test the read_file_hook
unittests.test_read_file_hook(read_file_hook)

<a id="6"></a>

## 6 - Creating the Crew

Now that all the elements are in place, you can define the Crew and kick it off to get the assessment for your PR.

<a id="ex7"></a>

### Exercise 7: Define the Crew
In this step, you will define the crew. You need to set up agents and tasks, just like in the previous module, but this time you will also need to add memory. Adding memory enables agents to remember previously identified security vulnerabilities and coding patterns across multiple pull requests, improving consistency and allowing them to recognize recurring issues without starting from scratch each time.

In [33]:
# GRADED CELL: Exercise 7

### START CODE HERE ###

# Create the code review crew
crew = Crew(
    # add the list of agents
    agents=[senior_developer, security_engineer, tech_lead],
    # add the list of tasks (in order of execution) 
    tasks=[analyze_code_quality, review_security, make_review_decision],
    # add memory to the crew
    memory=True,
    # add the before-kickoff hook to read the PR file
    # you need to pass a list with the hook function from Ex 8
    before_kickoff_callbacks= [read_file_hook]
)

### END CODE HERE ###


In [ ]:
# test the crew
unittests.test_crew(crew)

<a id="ex8"></a>

### Exercise 8: Kickoff the crew

If all tests from the previous exercises are passed, you are ready to kickoff the crew and save the results in a `dill` file. 

In [34]:
# GRADED CELL: Exercise 8

# define the file path for the PR file
file_path = "files/code_changes.txt" 

### START CODE HERE ###

# kickoff the crew
result = crew.kickoff(
    # add the inputs dictionary with the file path
    inputs={"file_path": file_path}
    )

### END CODE HERE ###

# Save the result to a dill file
with open('result.dill', 'wb') as f:
    dill.dump(result, f)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Developer                                                                                        │
│                                                                                                                 │
│  Task: 1. Review the code changes in the pull request. 2. Identify any style issues, potential bugs, or         │
│  maintainability concerns. 3. For each issue, decide if it's critical (must be fixed) or minor (suggestion).    │
│  PR content:  diff --git a/app/user_auth.py b/app/user_auth.py                                                  │
│  index 8f23c4d..b9e7f2a 100644                                                                                  │
│  --- a/app/user_auth.py                                                                                         │
│  +++ b/app/user_auth.py                                                                                         │
│  @@ -1,7 +1,32 @@                                                                                               │
│  +from datetime import datetime                                                                                 │
│  +import time                                                                                                   │
│  +                                                                                                              │
│   def authenticate_user(username, password):                                                                    │
│  +    # Check if username or password is empty                                                                  │
│  +    if not username or not password:                                                                          │
│  +        return False                                                                                          │
│  +                                                                                                              │
│  +    # Query the database for the user                                                                         │
│       user = db.query(f"SELECT * FROM users WHERE username = '{username}'")                                     │
│  +                                                                                                              │
│  +    # Verify the user exists and password matches                                                             │
│       if user and user.password == password:                                                                    │
│  +        # Set session variables                                                                               │
│           session['user_id'] = user.id                                                                          │
│  +        session['login_time'] = datetime.now()                                                                │
│  +                                                                                                              │
│  +        # Update last login timestamp                                                                         │
│  +        db.execute(f"UPDATE users SET last_login = NOW() WHERE id = {user.id}")                               │
│  +                                                                                                              │
│  +        print(f"User {username} logged in successfully")                                                      │
│           return True                                                                                           │
│  -    return False                                                                                              │
│  +    else:                                            

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Developer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "critical_issues": [                                                                                         │
│      "SQL injection risk due to use of string interpolation in SQL queries.",                                   │
│      "Password handling is insecure (plain text comparison, should use hashing)."                               │
│    ],                                                                                                           │
│    "minor_issues": [                                                                                            │
│      "Consider using a logging framework instead of print statements for audit trails.",                        │
│      "Use parameterized queries instead of raw SQL to enhance security and readability.",                       │
│      "Consider handling exceptions for database queries to avoid application crashes."                          │
│    ],                                                                                                           │
│    "reasoning": "The critical issues identified pose significant security risks, such as SQL injection          │
│  vulnerabilities and insecure password handling practices, which must be addressed before code approval. The    │
│  minor issues, while not immediately harmful, could enhance the maintainability and security posture of the     │
│  codebase if addressed. The use of logging instead of print statements promotes better practices for tracking   │
│  application behavior, while adopting parameterized queries would further enhance security. Exception handling  │
│  for database interactions would improve application robustness by managing potential errors gracefully."       │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Security Engineer                                                                                       │
│                                                                                                                 │
│  Task: 1. Review the findings from both the code quality and security analyses. 2. Determine if the pull        │
│  request can be automatically approved based on these findings. 3. If approval is not possible, decide whether  │
│  to request specific changes from the developer or escalate to human review. 4. Provide a clear explanation of  │
│  your decision and any required next steps. PR content:  diff --git a/app/user_auth.py b/app/user_auth.py       │
│  index 8f23c4d..b9e7f2a 100644                                                                                  │
│  --- a/app/user_auth.py                                                                                         │
│  +++ b/app/user_auth.py                                                                                         │
│  @@ -1,7 +1,32 @@                                                                                               │
│  +from datetime import datetime                                                                                 │
│  +import time                                                                                                   │
│  +                                                                                                              │
│   def authenticate_user(username, password):                                                                    │
│  +    # Check if username or password is empty                                                                  │
│  +    if not username or not password:                                                                          │
│  +        return False                                                                                          │
│  +                                                                                                              │
│  +    # Query the database for the user                                                                         │
│       user = db.query(f"SELECT * FROM users WHERE username = '{username}'")                                     │
│  +                                                                                                              │
│  +    # Verify the user exists and password matches                                                             │
│       if user and user.password == password:                                                                    │
│  +        # Set session variables                                                                               │
│           session['user_id'] = user.id                                                                          │
│  +        session['login_time'] = datetime.now()                                                                │
│  +                                                                                                              │
│  +        # Update last login timestamp                                                                         │
│  +        db.execute(f"UPDATE users SET last_login = NOW() WHERE id = {user.id}")                               │
│  +                                                                                                              │
│  +        print(f"User {username} logged in successfully")                                                      │
│           return True                                                                                           │
│  -    return False                                     

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Security Engineer                                                                                       │
│                                                                                                                 │
│  Thought: I need to evaluate the security vulnerabilities present in the user authentication pull request and   │
│  structure my findings accordingly.                                                                             │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "website_url": "http://example.com/user_auth.py"                                                             │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Security Engineer                                                                                       │
│                                                                                                                 │
│  Task: 1. Review the findings from both the code quality and security analyses. 2. Determine if the pull        │
│  request can be automatically approved based on these findings. 3. If approval is not possible, decide whether  │
│  to request specific changes from the developer or escalate to human review. 4. Provide a clear explanation of  │
│  your decision and any required next steps. PR content:  diff --git a/app/user_auth.py b/app/user_auth.py       │
│  index 8f23c4d..b9e7f2a 100644                                                                                  │
│  --- a/app/user_auth.py                                                                                         │
│  +++ b/app/user_auth.py                                                                                         │
│  @@ -1,7 +1,32 @@                                                                                               │
│  +from datetime import datetime                                                                                 │
│  +import time                                                                                                   │
│  +                                                                                                              │
│   def authenticate_user(username, password):                                                                    │
│  +    # Check if username or password is empty                                                                  │
│  +    if not username or not password:                                                                          │
│  +        return False                                                                                          │
│  +                                                                                                              │
│  +    # Query the database for the user                                                                         │
│       user = db.query(f"SELECT * FROM users WHERE username = '{username}'")                                     │
│  +                                                                                                              │
│  +    # Verify the user exists and password matches                                                             │
│       if user and user.password == password:                                                                    │
│  +        # Set session variables                                                                               │
│           session['user_id'] = user.id                                                                          │
│  +        session['login_time'] = datetime.now()                                                                │
│  +                                                                                                              │
│  +        # Update last login timestamp                                                                         │
│  +        db.execute(f"UPDATE users SET last_login = NOW() WHERE id = {user.id}")                               │
│  +                                                                                                              │
│  +        print(f"User {username} logged in successfully")                                                      │
│           return True                                                                                           │
│  -    return False                                     

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Security Engineer                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "security_vulnerabilities": [                                                                                │
│      {                                                                                                          │
│        "description": "SQL injection risk due to use of string interpolation in SQL queries.",                  │
│        "risk_level": "high",                                                                                    │
│        "evidence": "The SQL query in the authenticate_user function uses string interpolation for the           │
│  username, allowing for potential injection attacks."                                                           │
│      },                                                                                                         │
│      {                                                                                                          │
│        "description": "Password handling is insecure (plain text comparison, should use hashing).",             │
│        "risk_level": "high",                                                                                    │
│        "evidence": "The password check uses plain text comparison, which does not protect passwords from        │
│  attacks in case of data exposure."                                                                             │
│      }                                                                                                          │
│    ],                                                                                                           │
│    "blocking": true,                                                                                            │
│    "highest_risk": "high",                                                                                      │
│    "security_recommendations": [                                                                                │
│      "Use parameterized queries to prevent SQL injection.",                                                     │
│      "Implement password hashing and comparison using secure hashing algorithms (e.g., bcrypt).",               │
│      "Consider using a logging framework instead of print statements for better audit trails.",                 │
│      "Handle exceptions for database queries to avoid application crashes."                                     │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Lead                                                                                               │
│                                                                                                                 │
│  Task: 1. Based on the results from 'analyze_code_quality' and 'review_security', make a final review           │
│  decision. 2. Decide whether to approve the changes, request changes, or escalate to human review. 3.           │
│  Summarize the key findings from both analyses to support your decision. 4. If requesting changes, provide a    │
│  clear list of what must be fixed. 5. If approving, include appropriate comments to accompany the approval. 6.  │
│  If escalating, explain why human review is necessary. 7. Include any additional recommendations or             │
│  observations. PR content:  diff --git a/app/user_auth.py b/app/user_auth.py                                    │
│  index 8f23c4d..b9e7f2a 100644                                                                                  │
│  --- a/app/user_auth.py                                                                                         │
│  +++ b/app/user_auth.py                                                                                         │
│  @@ -1,7 +1,32 @@                                                                                               │
│  +from datetime import datetime                                                                                 │
│  +import time                                                                                                   │
│  +                                                                                                              │
│   def authenticate_user(username, password):                                                                    │
│  +    # Check if username or password is empty                                                                  │
│  +    if not username or not password:                                                                          │
│  +        return False                                                                                          │
│  +                                                                                                              │
│  +    # Query the database for the user                                                                         │
│       user = db.query(f"SELECT * FROM users WHERE username = '{username}'")                                     │
│  +                                                                                                              │
│  +    # Verify the user exists and password matches                                                             │
│       if user and user.password == password:                                                                    │
│  +        # Set session variables                                                                               │
│           session['user_id'] = user.id                                                                          │
│  +        session['login_time'] = datetime.now()                                                                │
│  +                                                                                                              │
│  +        # Update last login timestamp                                                                         │
│  +        db.execute(f"UPDATE users SET last_login = NOW() WHERE id = {user.id}")                               │
│  +                                                                                                              │
│  +        print(f"User {username} logged in successfull

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Lead                                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Final Code Review Decision                                                                                   │
│                                                                                                                 │
│  ## Final Decision: Request Changes                                                                             │
│                                                                                                                 │
│  ### Key Findings                                                                                               │
│  1. **Critical Issues:**                                                                                        │
│     - **SQL Injection Risk:** The code uses string interpolation to construct SQL queries, exposing it to       │
│  potential SQL injection attacks. For example, the `authenticate_user` function constructs a query with user    │
│  input directly, which is insecure.                                                                             │
│     - **Insecure Password Handling:** The current implementation compares plaintext passwords directly, which   │
│  is a significant security risk. Passwords should be hashed using secure algorithms like bcrypt.                │
│                                                                                                                 │
│  2. **Minor Issues:**                                                                                           │
│     - **Logging Practices:** The use of `print` statements for logging is not recommended. It's advisable to    │
│  use a logging framework for better auditability and maintenance.                                               │
│     - **Raw SQL Queries:** The code does not use parameterized queries, which would enhance both security and   │
│  readability.                                                                                                   │
│     - **Error Handling:** There is a lack of exception handling when accessing the database, which could lead   │
│  to application crashes without graceful error management.                                                      │
│                                                                                                                 │
│  ### Required Changes                                                                                           │
│  To ensure the security and quality of the code, the following changes must be made:                            │
│  - Refactor the SQL queries to use parameterized queries to prevent SQL injection. For example, replace:        │
│    ```python                                                                                                    │
│    user = db.query(f"SELECT * FROM users WHERE username = '{username}'")                                        │
│    ```                                                                                                          │
│    with a parameterized query approach.                                                                         │
│                                                                                                                 │
│  - Implement password hashing using a secure method bef

┌───────────────────────── Trace Batch Finalization ──────────────────────────┐
│ ✅ Trace batch finalized with session ID:                                   │
│ 0adc2863-217c-4636-a37e-5497a2ea827e                                        │
│                                                                             │
│ 🔗 View here:                                                               │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/0adc2863-217c-46 │
│ 36-a37e-5497a2ea827e?access_code=TRACE-6a46c8b8ed                           │
│ 🔑 Access Code: TRACE-6a46c8b8ed                                            │
└─────────────────────────────────────────────────────────────────────────────┘


Let's check out the final report!

In [35]:
# clean the markdown output, in case the report is wrapped with code fences
clean_report = clean_markdown(result.raw)

# display the final report as markdown
display(Markdown(clean_report))

# Final Code Review Decision

## Final Decision: Request Changes

### Key Findings
1. **Critical Issues:**
   - **SQL Injection Risk:** The code uses string interpolation to construct SQL queries, exposing it to potential SQL injection attacks. For example, the `authenticate_user` function constructs a query with user input directly, which is insecure.
   - **Insecure Password Handling:** The current implementation compares plaintext passwords directly, which is a significant security risk. Passwords should be hashed using secure algorithms like bcrypt.

2. **Minor Issues:**
   - **Logging Practices:** The use of `print` statements for logging is not recommended. It's advisable to use a logging framework for better auditability and maintenance.
   - **Raw SQL Queries:** The code does not use parameterized queries, which would enhance both security and readability.
   - **Error Handling:** There is a lack of exception handling when accessing the database, which could lead to application crashes without graceful error management.

### Required Changes
To ensure the security and quality of the code, the following changes must be made:
- Refactor the SQL queries to use parameterized queries to prevent SQL injection. For example, replace:
  ```python
  user = db.query(f"SELECT * FROM users WHERE username = '{username}'")
  ```
  with a parameterized query approach.
  
- Implement password hashing using a secure method before comparing passwords. For instance, use something like:
  ```python
  if user and check_password_hash(user.hashed_password, password):
  ```
  
- Replace `print` statements with a proper logging framework:
  ```python
  import logging
  logging.basicConfig(level=logging.INFO)
  logging.info(f"User {username} logged in successfully")
  ```

- Add exception handling around database queries to manage potential errors.

### Additional Recommendations
- It may be beneficial to conduct regular security training for development teams to raise awareness of best practices for secure coding.
- Implement unit tests to cover authentication functionalities to ensure stability and security over time.

Please address the above changes before resubmitting the pull request for review.

You reached the end of the assignment. At this point you are ready to submit for grading.

After submitting and being satisfied with your grade you can take some time to experiment changing the guardrails, or adding new URLs to the Website Reading Tool, or even upload a different pull request file with code differences. Don't be afraid to shake things up!